### ベイズ推論の線形回帰

In [ ]:
using Distributions
using Plots
gr()

In [ ]:
M = 4 # dimensions
N = 100
x = range(-1, 1, length=N)

In [ ]:
lambda = 10
epsilon = Normal(0, lambda^(-1))

ep = rand(epsilon, N)
plot(ep, seriestype=:scatter)

In [ ]:
# weight
m = zeros(M)
l_lambda = zeros(M, M)

for i in 1:M
    l_lambda[i, i] = 1
end

y = []
for i in 1:N
    input = [1, x[i], x[i]^2, x[i]^3]
    w = []
    y_buff = 0
    for j in 1:M
        dist_w = Normal(m[j], l_lambda[j, j]^(-1))
        w = pdf.(dist_w, x[i])
        y_buff += (input[j] * w)
    end
    
    dist_y = Normal(y_buff, lambda^(-1))
    push!(y, rand(dist_y, 1)[1]+ep[i])
end

# r = range(-1, 1, length=100)
plot(x, y, seriestype=:scatter)